***
### Part 1 of the Assignment

In [242]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')

Libraries imported.


### Load the data into a pandas dataframe

<font color=blue>**Assumption: The relevant table is the first table in the HTML file**</font>

In [243]:
# Use the pandas read_html method to read the tables in the Wikipedia page into a list of DataFrames
torontoDataList = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

# The data we want is in the first DataFrame
dfToronto = torontoDataList[0]

# Let's have a look at the DataFrame
dfToronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [244]:
# Create an empty DataFrame that we'll build up as we process the data
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [245]:
# For efficiency, create a list of dictionary entries containing the processed data,
# then make a DataFrame from that. This avoids the inefficiency of DataFrame.append,
# which creates a new DataFrame each call
rows_list = []

### Iterate over the DataFrame, processing the data

**panda DataFrames can get very slow if you keep 
appending rows to them, so we will instead build
a list of lists, called rows_list. 
Each row in rows_list will itself be a list
of three entries: postcode, borough, and neighborhood.
To merge data with the same postal code into 
one row, we'll use a dictionary that maps 
postal code to the index (row number) 
of that postal code's data in rows_list. 
That way, we can retrieve the data we have so far
for that postal code, and add the new neighborhood to it.
When we've processed all the input data, 
we'll convert rows_list into a pandas DataFrame.**


In [246]:
# To enable us to retrieve rows from rows_list using postalcode as a key, create a dictionary
# The key will be the postal code, and the value will be the index of that postal code's row in rows_list
rowsDict = {}

# Iterate over the rows in the DataFrame using itertuples because it preserves the columns' data types
for row in dfToronto.itertuples():
    borough = row.Borough
    
    # Per the assignment, ignore rows where the Borough is 'Not assigned'
    if borough != 'Not assigned':
        nIndex = row.Index
        postcode = row.Postcode
        neighborhood = row.Neighbourhood
        
        # If no neighborhood assigned, use the borough
        if neighborhood == 'Not assigned':
            neighborhood = borough
        #print(postcode, neighborhood)
        
        # See if we already have a dictionary entry in rows_list for this postal code
        if (postcode in rowsDict):
            # We've already seen this postal code, so append the new neighborhood to its data
            rowsListKey = rowsDict.get(postcode)
            rowData = rows_list[rowsListKey]
            rowData = [rowData[0], rowData[1], rowData[2] + ', ' + neighborhood]
            # Update rows_list with the new data
            rows_list[rowsListKey] = rowData
            #print(rowData)
        else:
            # We have not seen this postal code, so create a new row list of its data
            rowData = [postcode, borough, neighborhood]
            rows_list.append(rowData)
            rowsDict[postcode] = len(rows_list)-1
            #print(rowsDict)
            
# Now that we've got all the data in rows_list, make a DataFrame from it
dfProcessed = pd.DataFrame(rows_list)
dfProcessed.columns = ['PostalCode', 'Borough', 'Neighborhood']
#dfProcessed

In [247]:
# Print the number of rows in the DataFrame
print('The DataFrame has ' + str(dfProcessed.shape[0]) + ' rows')

The DataFrame has 103 rows


***
### Start of the 2nd Part of the Assignment
**I installed geocoder and called it, but it never worked, so I'm using the CSV file with the latitude/longitude**

In [248]:
# I could not get geocoder to work, so  use the pandas read_csv
# method to read the postal code -> Lat/Long data file
dfLatLong = pd.read_csv('https://cocl.us/Geospatial_data')

**Loop over rows_list, adding the latitude and longitude to each row
Look up the lat/long from the DataFrame we created from the CSV file**

In [249]:
nRowCounter = 0
for rowData in rows_list:
    latLongData = dfLatLong.loc[dfLatLong['Postal Code'] == rowData[0]]
    latitude = latLongData.iat[0, 1]
    longitude = latLongData.iat[0, 2]

    rowData = [rowData[0], rowData[1], rowData[2], latitude, longitude]
    nRowCounter = nRowCounter + 1
    
    # Now that we've added lat/long to the list element, update the list element in the list
    rowsListKey = rowsDict.get(rowData[0])
    rows_list[rowsListKey] = rowData
    
print('Processed ' + str(nRowCounter) + ' rows')

Processed 103 rows


**Create a DataFrame from rows_list, then names its columns**

In [250]:
dfProcessed2 = pd.DataFrame(rows_list)
dfProcessed2.columns = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']
#dfProcessed2

***
### Part 3 of the Assignment

In [251]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [252]:
# Import folium
#!conda install -c conda-forge folium=0.5.0 --yes

In [253]:
# The code was removed by Watson Studio for sharing.

In [254]:
# I have a JSON file containing Toronto neighborhood data as a data asset in Watson Studio.
# To get it into a json object here in python, I first have to read it
# in as a TextIOWrapper, then convert that to a string, then use
# the json library's 'loads' function to convert the string to a
# json object
import json
import io

# Read the file
wrapper = io.TextIOWrapper(project.get_file("simple.geojson"), encoding='utf-8')

# Convert it to a string
jsonString = wrapper.read()

# Convert the string to a json object
json_data = json.loads(jsonString)
#print(json_data)

In [255]:
# Extract the 'features' data from the JSON as a string
neighborhoods_data = json_data['features']

# Take a look and see what we've got
neighborhoods_data[0]

{'type': 'Feature',
 'properties': {'DAUID': '35200879',
  'PRUID': '35',
  'CSDUID': '3520005',
  'HOODNUM': 81,
  'HOOD': 'Trinity-Bellwoods',
  'FULLHOOD': 'Trinity-Bellwoods (81)'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-79.40428280044927, 43.64797961606815],
    [-79.403956753622, 43.64718271074494],
    [-79.42236786578222, 43.643467621011894],
    [-79.42640543946513, 43.65360764326518],
    [-79.41868792113178, 43.65521730993704],
    [-79.41769878521191, 43.65524323486715],
    [-79.41514736685951, 43.65496322517198],
    [-79.40767889826175, 43.65646442447146],
    [-79.40428280044927, 43.64797961606815]]]}}

#### Hmm, seems like the polygonal coordinates are going to be a problem down the road when we want to look up nearby venues on foursquare. I think we want the center of the neighborhood, so we need to find a centroid of the polygon's points, and each polygon may have a different number of points

In [256]:
# This is what one polygon's data looks like
neighborhoods_data[0]['geometry']['coordinates'][0]

[[-79.40428280044927, 43.64797961606815],
 [-79.403956753622, 43.64718271074494],
 [-79.42236786578222, 43.643467621011894],
 [-79.42640543946513, 43.65360764326518],
 [-79.41868792113178, 43.65521730993704],
 [-79.41769878521191, 43.65524323486715],
 [-79.41514736685951, 43.65496322517198],
 [-79.40767889826175, 43.65646442447146],
 [-79.40428280044927, 43.64797961606815]]

#### numpy to the rescue. Let's define a function that uses numpy to calculate the centroid of a varying number of points. Because polygons are not regular shapes, the calculated centroid may not be the exact geogrphic center, but it's close enough for our purposes. We could use a specialized library to get the exact center, but I don't think that's necessary in order to find nearby coffee shops, etc.

In [257]:
import numpy as np

def centeroidnp(arr):
    length, dim = arr.shape
    return np.array([np.sum(arr[:, i])/length for i in range(dim)])

In [258]:
# Let's see if it works; the resulting centroid should be in the "neighborhood" of -79.4, +43.6
c = neighborhoods_data[0]['geometry']['coordinates'][0]
centeroidnp(np.asarray(c))

array([-79.41338985,  43.65134504])

In [259]:
# Ok, now that we've figured out how to convert a neighborhood's polygon boundary to a centroid,
# we can convert the JSON data to a pandas DataFrame.

# define the dataframe columns
column_names = ['Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [260]:
# Now we can loop through the JSON data and build up the DataFrame
nLineCounter = 0
for data in neighborhoods_data:
    neighborhood_name = data['properties']['HOOD']
    neighborhood_geometry = data['geometry']
    neighborhood_geometry_coords = neighborhood_geometry['coordinates']
    if (len(neighborhood_geometry_coords[0]) == 1):
        coords = neighborhood_geometry_coords[0][0]
    else:
        coords = neighborhood_geometry_coords[0]

    # Print some output so we know it's working
    #print(str(nLineCounter) + ' ' + neighborhood_name)
    nLineCounter = nLineCounter + 1
    
    # Find the approximate center of the neighborhood
    neighborhood_lat = centeroidnp(np.asarray(coords))[1]
    neighborhood_lon = centeroidnp(np.asarray(coords))[0]
    
    neighborhoods = neighborhoods.append({'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, 
                                         ignore_index=True)

In [261]:
# Find the geographical center of Toronto
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [262]:
# Now make a map of Toronto
# create map of New York using latitude and longitude values
import folium # map rendering library

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [263]:
# Now let's deal with just neighborhoods that have 'Toronto' in their name
toronto_data_subset = neighborhoods[neighborhoods['Neighborhood'].str.contains('Toronto')].reset_index(drop=True)
toronto_data_subset.head()

,Neighborhood,Latitude,Longitude
0,New Toronto,43.592622,-79.510767
1,Danforth Village - Toronto,43.684570,-79.326172


### Use the next line to choose whether to operate on all the Toronto neighborhoods, or just the ones with 'Toronto' in the name

In [264]:
# UNCOMMENT ONE OF THE LINES BELOW

# The next line will make all subsequent code deal only with neighborhoods with 'Toronto' in the name
toronto2_data = toronto_data_subset

# The next line will make all subsequent code deal with all Toronto neighborhoods
#toronto2_data = neighborhoods

In [265]:
# The code was removed by Watson Studio for sharing.

In [266]:
# Let's explore the first "Toronto" neighbhood
neighborhood_latitude = toronto2_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto2_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = toronto2_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of New Toronto are 43.59262155426167, -79.51076671603133.


#### The next few blocks of code come directly from the lab on exploring NYC neighborhoods. Let's get the top 100 venues within 500 meters of the center of the first neighborhood

In [267]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}\
&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)


In [268]:
import requests # library to handle requests

results = requests.get(url).json()
#results

In [269]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [270]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Japango,Sushi Restaurant,43.655268,-79.385165
2,Textile Museum of Canada,Art Museum,43.654396,-79.386500
3,Poke Guys,Poke Place,43.654895,-79.385052
4,Sansotei Ramen 三草亭,Ramen Restaurant,43.655157,-79.386501


In [271]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

72 venues were returned by Foursquare.


### More code taken from the NYC lab, it retrieves nearby venues for any neighborhood

In [272]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Let's get nearby venues for every Toronto neighborhood and see how many venues each one has

In [273]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )


Trinity-Bellwoods
West Humber-Clairville
Mount Olive-Silverstone-Jamestown
Humber Summit
Thistletown-Beaumond Heights
Humbermede
Rexdale-Kipling
Elms-Old Rexdale
Pelmo Park-Humberlea
Downsview-Roding-CFB
Kingsview Village-The Westway
Weston
Rustic
Humber Heights-Westmount
Brookhaven-Amesbury
Mount Dennis
Willowridge-Martingrove-Richview
Princess-Rosethorn
Eringate-Centennial-West Deane
Edenbridge-Humber Valley
Rockcliffe-Smythe
Markland Woods
Etobicoke West Mall
Islington-City Centre West
Alderwood
Long Branch
New Toronto
Mimico
Stonegate-Queensway
Kingsway South
Lambton Baby Point
High Park-Swansea
Runnymede-Bloor West Village
High Park North
Junction Area
Roncesvalles
South Parkdale
Dufferin Grove
Little Portugal
Dovercourt-Wallace Emerson-Juncti
Niagara
Weston-Pellam Park
Palmerston-Little Italy
University
Kensington-Chinatown
Waterfront Communities-The Island
Annex
Wychwood
Corsa Italia-Davenport
Oakwood-Vaughan
Humewood-Cedarvale
Casa Loma
Forest Hill South
Yonge-St.Clair
Bay Stre

In [274]:
# How many venues in each 'hood?
#toronto_venues.groupby('Neighborhood').count()

In [275]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 277 uniques categories.


In [276]:
#toronto_venues

In [277]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.head()

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# The Neighborhood column is in the middle of the DataFrame, because the columns
# come out ordered alphabetically. We need to move Neighborhood to be the first column
# Make a list of columns that does not include Neighborhood
columnList = [col for col in toronto_onehot.columns if col not in ['Neighborhood']]
columnList.insert(0, 'Neighborhood')
toronto_onehot = toronto_onehot[columnList]
toronto_onehot.head()

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,College Theater,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Public Art,Racecourse,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tree,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Trinity-Bellwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Group rows by neighborhood, and take the mean of the frequency of occurrence of each category


In [278]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
#toronto_grouped

#### From the NYC lab, borrow code to return the top N venues for a neighborhood

In [279]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### From the NYC lab, also borrow code to find the top 10 venues in each neighborhood, and build a DataFrame of that data

In [280]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt North,IT Services,Park,Zoo Exhibit,Ethiopian Restaurant,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store
1,Agincourt South-Malvern West,Pool Hall,Badminton Court,Motorcycle Shop,Lounge,Skating Rink,Breakfast Spot,Convenience Store,Cosmetics Shop,Festival,Comic Shop
2,Alderwood,Pub,Pool,Coffee Shop,Pizza Place,Gym,Gas Station,Sandwich Place,Skating Rink,Pharmacy,Dive Bar
3,Annex,Italian Restaurant,Pub,Café,American Restaurant,Mexican Restaurant,Mediterranean Restaurant,Coffee Shop,Shoe Repair,Electronics Store,Shopping Mall
4,Banbury-Don Mills,Cantonese Restaurant,Gourmet Shop,Movie Theater,Bubble Tea Shop,Liquor Store,Sandwich Place,Pet Store,Pharmacy,Spa,Cosmetics Shop


### Do some clustering

In [281]:
from sklearn.cluster import KMeans 

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 0, 0, 2, 0, 0, 0, 0], dtype=int32)

In [282]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto2_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New Toronto,43.592622,-79.510767,0,Park,Pool,Harbor / Marina,Skating Rink,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
1,Danforth Village - Toronto,43.684570,-79.326172,0,Café,Ethiopian Restaurant,Bar,Coffee Shop,American Restaurant,Middle Eastern Restaurant,Beer Bar,Bowling Alley,Metro Station,Mexican Restaurant


### The NYC lab added a single marker for each location. I'm going a little further and adding true cluster markers, which you can click on to see the top venues for each location

In [283]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from folium.plugins import MarkerCluster

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#venueColumns = list(toronto_merged.columns)[4:]
venueColumns = list(toronto_merged.columns)
#print(venueColumns)
                     

In [284]:
# add cluster markers to the map, with a cluster child for each top venue
# iterate over the rows in the DataFrame
for clusterRow in toronto_merged.itertuples():
    hood = clusterRow.Neighborhood
    lat = clusterRow.Latitude
    lon = clusterRow.Longitude

# Iterate over the top venues, adding each of them to a single cluster marker
    mc = MarkerCluster()
    for nCol in range(5, len(venueColumns)+1):
        venueType = clusterRow[nCol]
#        print(str(nCol-4) + ' ' + venueType) 
        thePopup = folium.Popup(venueType)
        mc.add_child(folium.Marker(location=[lat,  lon], popup=thePopup))
    map_clusters.add_child(mc)

map_clusters